# 리뷰 정리
- 공백 리뷰 버리기
- 무의미 리뷰 버리기(Jimin's Rule)
(- 리뷰 길이)
- 처리 완료된 리뷰 저장 ** String 그대로 **  
    Reviews_merged_clean.csv  
    Reviews_clean.txt  
- BPE로 처리 완료된 리뷰 저장  
    Reviews_merged_processed.csv



### TODO
1. 리뷰 버리는 기준 보완하기: 진짜 필요 없는 애만 버리도록!
2. 처리 완료된 리뷰 저장 ** String 그대로 **  
    Reviews_meged_clean.csv  
    Reviews_clean.txt 


In [1]:
import pandas as pd
from konlpy.tag import Komoran, Twitter
from tqdm import tqdm_notebook

pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 2000)

### Review csv 파일 불러오기

In [8]:
def read_from_csv(file):
    """
    input : path/file.csv
    return: pd.DataFrame with None review dropped    
    """
    df = pd.read_csv(file, index_col=0)
    df = df[df['Review'].notnull()]
    return df 

In [9]:
file = '../crawling/Reviews_csv/reviews_merged.csv'
reviews_all_df = read_from_csv(file)
reviews_all_df.head()

,Id,Rating,Review
0,2301428333,추천,기여운 동전지갑♡ 복실복실 기여운 털동전지갑입니다. 가방 키링으로도 괜찮구요~ 열쇠...
1,2301428333,추천,기염 깜찍^^ 털이 보슬보슬.하구요~~^^ 귀여워요. 동전이랑 립스틱정도 넣고 다니...
2,2301428333,적극추천,귀여우용 제가 쓰려고 샀는데 카드가 안들어가길래 당황햇어요ㅠㅠ 그래서 어머니 키링으...
3,2301428333,적극추천,ㅇ선물 칭구 선물이라 일칙 주려구했는데 배송이 좀 걸렸지만 판매자분이 친절해서 기다...
4,2301428333,적극추천,배송 ㅠ 배송 일칙 받을수 잇다구해서 시켯는데 늦어졌음 ㅜㅜ 그래도 판매자분이 친절...


### 공백 리뷰 버리기

In [10]:
def drop_empty_reviews(df):
    before = len(df)
    review_series = df['Review']
    review_list_stripped = [item.strip() for item in review_series]
    df['Review'] = review_list_stripped
    df = df[df['Review'] != '']
    after = len(df)
    print("dropped: %d\n" % (before-after))
    return df

In [11]:
reviews_all_df = drop_empty_reviews(reviews_all_df)
reviews_all_df.info()

dropped: 2921

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264005 entries, 0 to 108934
Data columns (total 3 columns):
Id        264005 non-null int64
Rating    264005 non-null object
Review    264005 non-null object
dtypes: int64(1), object(2)
memory usage: 8.1+ MB


### 중복 리뷰 버리기

In [12]:
def drop_duplicate_reviews(df):
    before = len(df)
    df = df.drop_duplicates()
    after = len(df)
    print("dropped: %d\n" % (before-after))
    return df

In [13]:
reviews_all_df = drop_duplicate_reviews(reviews_all_df)
reviews_all_df.head()

dropped: 29034



,Id,Rating,Review
0,2301428333,추천,기여운 동전지갑♡ 복실복실 기여운 털동전지갑입니다. 가방 키링으로도 괜찮구요~ 열쇠...
1,2301428333,추천,기염 깜찍^^ 털이 보슬보슬.하구요~~^^ 귀여워요. 동전이랑 립스틱정도 넣고 다니...
2,2301428333,적극추천,귀여우용 제가 쓰려고 샀는데 카드가 안들어가길래 당황햇어요ㅠㅠ 그래서 어머니 키링으...
3,2301428333,적극추천,ㅇ선물 칭구 선물이라 일칙 주려구했는데 배송이 좀 걸렸지만 판매자분이 친절해서 기다...
4,2301428333,적극추천,배송 ㅠ 배송 일칙 받을수 잇다구해서 시켯는데 늦어졌음 ㅜㅜ 그래도 판매자분이 친절...


### 리뷰에 있는 개행문자('\n') 대체하기

In [14]:
def replace_newline(df):
    review_list = df['Review'].tolist()
    for idx, review in enumerate(review_list):
        review_list[idx] = review.replace("\n", "")
    df['Review'] = review_list
    return df

In [15]:
reviews_all_df = replace_newline(reviews_all_df)
reviews_all_df.head()

,Id,Rating,Review
0,2301428333,추천,기여운 동전지갑♡ 복실복실 기여운 털동전지갑입니다. 가방 키링으로도 괜찮구요~ 열쇠...
1,2301428333,추천,기염 깜찍^^ 털이 보슬보슬.하구요~~^^ 귀여워요. 동전이랑 립스틱정도 넣고 다니...
2,2301428333,적극추천,귀여우용 제가 쓰려고 샀는데 카드가 안들어가길래 당황햇어요ㅠㅠ 그래서 어머니 키링으...
3,2301428333,적극추천,ㅇ선물 칭구 선물이라 일칙 주려구했는데 배송이 좀 걸렸지만 판매자분이 친절해서 기다...
4,2301428333,적극추천,배송 ㅠ 배송 일칙 받을수 잇다구해서 시켯는데 늦어졌음 ㅜㅜ 그래도 판매자분이 친절...


### 리뷰 형태소 분석 및 버리기(기준: Jimin's rule)

In [16]:
def get_alphabet_ratio(sentence):
    num_alphabet = sum(item[1] == 'Alpha' for item in sentence)
    return num_alphabet / len(sentence)

def get_kparticle_num(sentence):
    return sum(item[1] == 'KoreanParticle' for item in sentence)

def pos_tag_reviews(df, norm=True):
    """
    pos tag reviews using twitter.pos(norm=True)
    """
    temp_df = df.copy()
    review_list = temp_df['Review'].tolist()
    tagged_review_list = []
    twitter = Twitter()    
    
    print("pos tagging reviews...")
    for review in tqdm_notebook(review_list):
        analyzed_result = twitter.pos(review, norm=norm)
        tagged_review_list.append(analyzed_result)
    temp_df['Review'] = tagged_review_list    
    return temp_df

def get_keep_mask(df):
    """
    drops useless reviews based on Twitter.pos result
    """
    df_tagged = pos_tag_reviews(df)
    review_list = df_tagged['Review'].tolist()
    to_keep = []

    print("dropping useless_reviews...")
    for review in review_list:
        num_koreanparticle = get_kparticle_num(review)
        ratio_alphabet = get_alphabet_ratio(review)
        decision = False if num_koreanparticle != 0 or ratio_alphabet == 1 else True
        to_keep.append(decision)
    print('Reduced from %d => %d (%.2f%%)' % (len(df), sum(to_keep), 
                                              (sum(to_keep)/len(df))*100))
    return to_keep

In [17]:
to_keep = get_keep_mask(reviews_all_df)
reviews_clean_df = reviews_all_df[to_keep]
reviews_clean_df.head()

pos tagging reviews...



dropping useless_reviews...
Reduced from 234971 => 166777 (70.98%)


,Id,Rating,Review
0,2301428333,추천,기여운 동전지갑♡ 복실복실 기여운 털동전지갑입니다. 가방 키링으로도 괜찮구요~ 열쇠...
1,2301428333,추천,기염 깜찍^^ 털이 보슬보슬.하구요~~^^ 귀여워요. 동전이랑 립스틱정도 넣고 다니...
5,2301428333,적극추천,너무 이뻐요 ^^ 사진으로 남겨보기는 처음이에요 근데 너무너무 만족스러워요 ^^ 너...
6,2301428333,적극추천,강추템 털빠짐이 좀 있긴한데 처음만 그러구 괜찮아요^.^ 판매자분도 너무 친절하시고...
7,2301428333,적극추천,너무 귀엽습니다! 아는 동생에게 선물할겸 커플로 하려고 검은색 두개로 샀는데 저도 ...


In [105]:
reviews_clean_df

,Id,Rating,Review
0,2301428333,추천,기여운 동전지갑♡ 복실복실 기여운 털동전지갑입니다. 가방 키링으로도 괜찮구요~ 열쇠...
1,2301428333,추천,기염 깜찍^^ 털이 보슬보슬.하구요~~^^ 귀여워요. 동전이랑 립스틱정도 넣고 다니...
5,2301428333,적극추천,너무 이뻐요 ^^ 사진으로 남겨보기는 처음이에요 근데 너무너무 만족스러워요 ^^ 너...
6,2301428333,적극추천,강추템 털빠짐이 좀 있긴한데 처음만 그러구 괜찮아요^.^ 판매자분도 너무 친절하시고...
7,2301428333,적극추천,너무 귀엽습니다! 아는 동생에게 선물할겸 커플로 하려고 검은색 두개로 샀는데 저도 ...
10,2301428333,만족,털 부드럽고 좋아요^.^
11,2301428333,만족,친구 선물용으로 줘써여 차키링으로 조아여
12,2301428333,만족,가방에 달고다니는중이에여
14,2301428333,만족,선물주려고 산건데.. 기냥 제가 다 쓰려구요 너무 이뻐요 ^^
15,2301428333,만족,진짜 너무너무너무 이뻐요! 꼭 구매추천!!


In [107]:
# 버리는 review들 들여다보기: 제대로 버리고 있나?
to_drop = [not decision for decision in to_keep]
reviews_dropped_df = reviews_all_df[to_drop]
reviews_dropped_df

,Id,Rating,Review
2,2301428333,적극추천,귀여우용 제가 쓰려고 샀는데 카드가 안들어가길래 당황햇어요ㅠㅠ 그래서 어머니 키링으...
3,2301428333,적극추천,ㅇ선물 칭구 선물이라 일칙 주려구했는데 배송이 좀 걸렸지만 판매자분이 친절해서 기다...
4,2301428333,적극추천,배송 ㅠ 배송 일칙 받을수 잇다구해서 시켯는데 늦어졌음 ㅜㅜ 그래도 판매자분이 친절...
8,2301428333,적극추천,너무 귀여워용 판매자분이 너무 친절하셔서 의리로 다른거도 주문해야할거같은기분.. 암...
9,2301428333,적극추천,만족 동전밖에안들어간다는거 알고삿어요 너무 귀엽네여ㅠㅠㅠ친구랑 하나씩하려고요 가방에...
13,2301428333,만족,ㅋㅋ귀여워욤
16,2301428333,만족,너무 귀엽고 털도 부드럽습니다.ㅠㅠ 아는동생이랑 커플로 하려구 검은색 두개를 샀는데...
17,2301428333,만족,너무 귀엽.ㅂㄷㄱㄴㅂ
19,2301428333,만족,만족해요 너무귀여워요ㅠㅠㅠ
23,2429054242,적극추천,ㅎㅎ 핑크로 샀구요 배송은 빨리왔어요 ㅋㅋㅋㅋ 실제로 보니 더이쁘고 동전도 안새나갈...


punctuation! 
~~~~~  
\* \*  
^^  
하트 이모티콘  


### Review plain text로 저장

In [ ]:
# review_list = reviews_clean_df['Review'].tolist()
# review_string = "\n".join(review_list)
# with open("../crawling/Reviews_csv/Reviews_merged_clean.txt", "w") as f:
#     f.write(review_string)
# print(review_string[:300])

### BPE 적용하기

In [14]:
# BPE learn-joint-bpe-and-vocab & apply-bpe 둘 다 된 상태에서!
with open('../crawling/Reviews_csv/Reviews_merged_processed.txt') as f:
    review_list = f.readlines()
reviews_clean_df['Review'] = review_list
reviews_clean_df.head()

/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Id,Rating,Review
0,2301428333,추천,기여@@ 운 동전@@ 지갑@@ ♡ 복@@ 실@@ 복@@ 실 기여@@ 운 털@@ 동전...
1,2301428333,추천,기@@ 염 깜찍@@ ^^ 털이 보@@ 슬@@ 보@@ 슬@@ .@@ 하구요@@ ~~^...
5,2301428333,적극추천,너무 이뻐요 ^^ 사진으로 남겨@@ 보@@ 기는 처음이에요 근데 너무너무 만족스러워...
6,2301428333,적극추천,강추@@ 템 털빠짐이 좀 있긴한데 처음@@ 만 그러@@ 구 괜찮아요@@ ^.^ 판매...
7,2301428333,적극추천,너무 귀엽@@ 습니다! 아는 동생@@ 에게 선물@@ 할@@ 겸 커플로 하려고 검은색...


### 파일 저장

In [15]:
reviews_clean_df.to_csv("../crawling/Reviews_csv/Reviews_merged_processed.csv", \
                        encoding='utf-8')